In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd
import  matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np


import torch
from torch.utils.data.dataloader import DataLoader

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import DefaultDataCollator
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from datasets import Dataset
from datasets import DatasetDict
import evaluate



from fintuned_use_parameter import fintuned_use_parameter

In [2]:
parameter = fintuned_use_parameter()

In [3]:
def multi_predict(text, tokenizer_, model_, label_list):
    #print(text)
    t_tokenize = tokenizer_(text, max_length=parameter.max_length, truncation=True, padding="max_length", return_tensors='pt').to(parameter.device)
    with torch.no_grad():
        output = model_(**t_tokenize).logits
        #print(output)
        preds = torch.where(output > 0, 1, 0)
    label = [[label_list[idx] for idx in range(len(pred)) if pred[idx] == 1] for pred in preds][0]
    label = '|'.join(label)
    return label

In [4]:
for file in parameter.use_list:
    file = os.path.join(parameter.use_data_dir, file)
    print("==========")
    print('file-name: ', file)
    try:
        file_pd = pd.read_excel(file)
    except FileNotFoundError:
        file_pd = pd.read_csv(file, encoding='cp932')
    for idx, (label_column_name, label_list) in enumerate(zip(parameter.label_column_name, parameter.label_list)):
        print('model-name: ', parameter.use_model_dir_list[idx])
        print('tokenizer-name: ',parameter.use_tokenizer[idx])
        tokenizer = AutoTokenizer.from_pretrained(parameter.use_tokenizer[idx])
        #ここのモデルは自前の場合は手作業で変更
        model = AutoModelForSequenceClassification.from_pretrained(parameter.use_model_dir_list[idx], num_labels=len(label_list),problem_type = "multi_label_classification").to(parameter.device)
        file_pd[label_column_name] = file_pd[parameter.text_column_name].apply(
            lambda x: multi_predict(str(x), 
                                    tokenizer_ = tokenizer, 
                                    model_= model,
                                    label_list = label_list)
            )
    print("==========")

file-name:  ./data/test/sokushin_test.xlsx
model-name:  ./result/sokushin
tokenizer-name:  nlp-waseda/roberta-base-japanese


/home/wakasugi/anaconda3/envs/transformers2023/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
file_pd

,ツイート,判定結果,促進軸
0,残念 旦那 の 実家 が 九州 南部 で 台風 回 バードストライク 回 と 滞在 が 延び...,その他,推奨
1,さっき の 東京 の 大雨 で 飯田橋 の 神田川 が 氾濫 危険 水位 途 に 階梯 浸水...,推奨|行動抑制,推奨|行動抑制
2,です ね 使い 切って から 購入 し ましょ 私 は 乾燥 肌 敏感 肌 です が スキン...,推奨,推奨
3,おはよう 御座います 台風 の 進路 に 注意 し つつ 笑顔 で 迎える 良い 週 末 を,推奨,推奨
4,千葉 の 台風 で ください した 方々 へ 送って 被災,願望,
...,...,...,...
3238,本日 ずーっと 晴天 気仙沼 日和 その 台風 一 過 の 割に 凄く 綺麗な 海 でした ...,推奨,
3239,たぶん ピーク 過ぎ たわ なかなか 元気 の ある 台風 や から そっち 行ったら みん...,推奨,推奨
3240,ピオ たん お は ピオニュース を 見て ビックリ して る よ 生き物 を 扱う お 仕...,その他,推奨
3241,土砂 災害 に 警戒 を 大型で 非常に 強い 台風 号 の 影響 で 宮崎 県 で は 土...,推奨,推奨
